#### This notebook generates `predictable.json.gz` and `predictable.bin`

This is part of the broader documentation for the `dorsal.file.utils.Predictable` class

The idea here is to create a sequence of random numbers which is fixed, to use in a separate class which simply cycles through the numbers based on some offset

The broader picture is a sampling algorithm which requires RNG, and must have the exact same output given a seed - but which must be cross-platform compatible

Scaling is used to avoid using float arithmetic, to ensure the end result can be ported to as many languages as possible

In [1]:
import gzip
import hashlib
import json
import struct

import numpy as np

In [2]:
scale = 10**18

In [3]:
np.__version__

'2.3.1'

In [4]:
np.random.seed(314159)
float_sequence = np.random.random(100_000).tolist()
integer_sequence = [int(i * scale) for i in float_sequence]

In [5]:
float_sequence[:10]

[0.8179233079975224,
 0.5510462969367323,
 0.4197753547567066,
 0.09869185474419295,
 0.8110207483765659,
 0.9673564038996015,
 0.09820669376309132,
 0.8018603712796477,
 0.6049021174289217,
 0.5847688110748427]

In [6]:
integer_sequence[:10]

[817923307997522432,
 551046296936732352,
 419775354756706560,
 98691854744192960,
 811020748376565888,
 967356403899601536,
 98206693763091328,
 801860371279647744,
 604902117428921728,
 584768811074842624]

Json copy

In [7]:
with gzip.open("predictable.json.gz", "wb") as fp:
    fp.write(json.dumps(integer_sequence).encode("utf-8"))

In [8]:
with gzip.open("predictable.json.gz") as fp:
    assert json.loads(fp.read()) == integer_sequence

Packed binary (struct)

In [9]:
format_string = f"<{len(integer_sequence)}Q"  # Q for 64-bit unsigned int; < for little-endian byte order
packed_data = struct.pack(format_string, *integer_sequence)
with open("predictable.bin", "wb") as fp:
    fp.write(packed_data)

In [10]:
with open("predictable.bin", "rb") as fp:
    unpacked = list(struct.unpack(format_string, fp.read()))

In [11]:
assert unpacked == integer_sequence

Generate hashes for validation

- The `predictable.bin` hash can be checked at load time for the Python library

In [12]:
with open("predictable.bin", "rb") as fp:
    bin_sha = hashlib.sha256(fp.read()).hexdigest()
bin_sha

'b1e601f5605b411e789a1d309216ef6b01331dbfb969616f05db9cbe4d641e73'

In [13]:
with open("predictable.json.gz", "rb") as fp:
    gold_sha = hashlib.sha256(fp.read()).hexdigest()
gold_sha

'373c91910d8e00bc24c97084f883a97510fbf83b944cdb3335c6aca7a3a5fd8b'